# ANITA Backend Development Guide

## 1. Introduction

This guide provides comprehensive instructions for developing new backend components, APIs, routes, WebSockets, and integrating with hardware for the ANITA (Advanced NFC, Identification & Technology Authentication) platform. It covers the creation of API endpoints, WebSocket communication, hardware integration, and ensuring compatibility with frontend requirements.

## 2. Development Environment Setup

### 2.1 Prerequisites

- ✅ Python 3.10+ installed
- ✅ FastAPI and Uvicorn installed
- ✅ PC/SC libraries for smartcard operations
- ✅ Code editor (VSCode recommended)
- ⬜ Linting and formatting tools configuration

### 2.2 Local Setup

In [ ]:
%%bash
# Clone the repository (if not already done)
git clone <repository-url>

# Navigate to project directory
cd anita/poc

# Create virtual environment (optional but recommended)
python -m venv venv
source venv/bin/activate  # On Windows: venv\Scripts\activate

# Install dependencies
pip install -r requirements.txt

# Start development server
python app.py
# or using uvicorn directly:
uvicorn asgi:app --reload

## 3. Project Structure

The ANITA backend is organized with a modular structure:

In [ ]:
%%markdown
anita/poc/
├── app.py                 # Main application entry point
├── asgi.py                # ASGI interface for production deployment
├── config.py              # Configuration management
├── backend/               # Core backend modules
│   ├── routes/            # API and WebSocket routes
│   │   ├── api/           # REST API endpoints
│   │   ├── websockets/    # WebSocket endpoints
│   │   └── ui/            # UI-related routes
│   ├── hardware/          # Hardware abstraction layers
│   ├── services/          # Business logic and services
│   └── utils/             # Utility functions and helpers
├── data/                  # Data storage and management
└── documentation/         # Project documentation

## 4. Creating New Routes/Endpoints

### 4.1 API Route Creation

#### 4.1.1 Create a new route file in the appropriate directory

In [ ]:
# backend/routes/api/your_feature.py
from fastapi import APIRouter, Depends, HTTPException
from pydantic import BaseModel
import logging

logger = logging.getLogger(__name__)
router = APIRouter()

class YourRequestModel(BaseModel):
    parameter1: str
    parameter2: int = None  # Optional parameter

@router.get("/your-endpoint", tags=["your-feature"])
async def get_your_feature():
    """Endpoint description."""
    try:
        # Implementation here
        return {"status": "success", "data": "Your data here"}
    except Exception as e:
        logger.error(f"Error in get_your_feature: {str(e)}")
        raise HTTPException(status_code=500, detail=f"Internal server error: {str(e)}")

@router.post("/your-endpoint", tags=["your-feature"])
async def create_your_feature(request: YourRequestModel):
    """Endpoint description."""
    try:
        # Implementation here
        return {"status": "success", "message": "Operation completed successfully"}
    except Exception as e:
        logger.error(f"Error in create_your_feature: {str(e)}")
        raise HTTPException(status_code=500, detail=f"Internal server error: {str(e)}")

#### 4.1.2 Include the router in the main application

In [ ]:
# In app.py
from backend.routes.api.your_feature import router as your_feature_router

# Include the router
app.include_router(your_feature_router, prefix="/api/your-feature", tags=["your-feature"])

### 4.2 Frontend Route Creation

For routes that serve HTML pages:

In [ ]:
@app.get("/your-feature", response_class=HTMLResponse)
async def your_feature_page(request: Request):
    """Renders the your-feature page."""
    return templates.TemplateResponse("your-feature.html", {"request": request})

## 5. Python Coding Standards

### 5.1 Naming Conventions

- ✅ Use snake_case for variables, functions, and methods
- ✅ Use PascalCase for classes and type definitions
- ✅ Use UPPERCASE for constants
- ✅ Prefix private variables/methods with underscore (_)

### 5.2 Documentation

- ✅ Include docstrings for all modules, classes, and functions
- ✅ Use type hints for function parameters and return values
- ✅ Include meaningful comments for complex logic
- ✅ Document API endpoints with clear descriptions

### 5.3 Error Handling

- ✅ Use try/except blocks for exception handling
- ✅ Log exceptions with appropriate levels (error, warning, info)
- ✅ Return standardized error responses from API endpoints
- ✅ Include specific error codes and messages for client handling

## 6. API Development

### 6.1 Request/Response Models

Define clear request and response models using Pydantic:

In [ ]:
from pydantic import BaseModel, Field
from typing import List, Optional

class DeviceInfo(BaseModel):
    id: str = Field(..., description="Unique device identifier")
    name: str = Field(..., description="Device name")
    type: str = Field(..., description="Device type")
    status: str = Field(..., description="Current status")

class DevicesResponse(BaseModel):
    status: str = Field(..., description="Operation status")
    message: str = Field(..., description="Status message")
    devices: List[DeviceInfo] = Field([], description="List of devices")

### 6.2 API Endpoint Structure

Follow a consistent pattern for endpoint implementation:

1. Input validation with Pydantic models
2. Authorization/authentication (when implemented)
3. Business logic processing
4. Standardized response formatting
5. Error handling

### 6.3 Response Formatting

Use a consistent response format:

In [ ]:
def standard_response(message: str, data=None, additional_fields=None):
    """Generate a standardized API response"""
    response = {
        "status": "success",
        "message": message
    }
    
    if data is not None:
        response["data"] = data
        
    if additional_fields:
        response.update(additional_fields)
        
    return response

def error_response(message: str, error_type: str = None, suggestion: str = None):
    """Generate a standardized error response"""
    response = {
        "status": "error",
        "message": message
    }
    
    if error_type:
        response["error_type"] = error_type
        
    if suggestion:
        response["suggestion"] = suggestion
        
    return response

## 7. WebSocket Implementation

### 7.1 Creating a WebSocket Endpoint

In [ ]:
@app.websocket("/ws/your-feature")
async def your_feature_websocket(websocket: WebSocket):
    await websocket.accept()
    try:
        logger.info("WebSocket connection established")
        # Send initial state or confirmation
        await websocket.send_json({
            "type": "connection_established",
            "payload": {"status": "connected"}
        })
        
        while True:
            # Receive and process messages
            data = await websocket.receive_json()
            await process_websocket_message(websocket, data)
            
    except WebSocketDisconnect:
        logger.info("WebSocket disconnected")
    except Exception as e:
        logger.error(f"WebSocket error: {str(e)}")
    finally:
        # Clean up resources if needed
        logger.info("WebSocket connection closed")

### 7.2 Message Processing

In [ ]:
async def process_websocket_message(websocket: WebSocket, data: dict):
    """Process incoming WebSocket messages"""
    try:
        # Extract message type and payload
        message_type = data.get("type")
        payload = data.get("payload", {})
        
        # Handle different message types
        if message_type == "get_active_devices":
            # Fetch active devices and send response
            devices = await get_devices()
            await websocket.send_json({
                "type": "device_list",
                "payload": {"devices": devices}
            })
        elif message_type == "start_tracking":
            # Start device tracking
            device_id = payload.get("device_id")
            # Implementation...
            await websocket.send_json({
                "type": "tracking_started",
                "payload": {"device_id": device_id}
            })
        else:
            # Unknown message type
            logger.warning(f"Unknown WebSocket message type: {message_type}")
            await websocket.send_json({
                "type": "error",
                "payload": {"message": f"Unknown message type: {message_type}"}
            })
    except Exception as e:
        logger.error(f"Error processing WebSocket message: {str(e)}")
        await websocket.send_json({
            "type": "error",
            "payload": {"message": "Internal server error"}
        })

### 7.3 WebSocket Message Format

Standardize on a consistent message format:

```json
{
  "type": "message_type",
  "payload": {
    "key1": "value1",
    "key2": "value2"
  }
}
```

## 8. Hardware Integration

### 8.1 Hardware Abstraction

Create abstraction layers for hardware devices:

In [ ]:
class DeviceManager:
    """Base class for hardware device management"""
    
    def __init__(self, config=None):
        self.config = config or {}
        self.devices = {}
        self.is_initialized = False
        self.logger = logging.getLogger(__name__)
    
    async def initialize(self):
        """Initialize the device manager"""
        try:
            # Implementation...
            self.is_initialized = True
            return True
        except Exception as e:
            self.logger.error(f"Error initializing device manager: {str(e)}")
            return False
    
    async def get_devices(self):
        """Get list of available devices"""
        # Implementation...
        pass
    
    async def connect_device(self, device_id):
        """Connect to a specific device"""
        # Implementation...
        pass

### 8.2 Error Handling for Hardware Operations

In [ ]:
def handle_device_exceptions(func):
    """Decorator to handle hardware-related exceptions"""
    @functools.wraps(func)
    async def wrapper(*args, **kwargs):
        try:
            return await func(*args, **kwargs)
        except DeviceNotFoundError as e:
            logger.error(f"Device not found: {str(e)}")
            return JSONResponse(
                status_code=404,
                content=error_response(
                    message=str(e),
                    error_type="DeviceNotFoundError",
                    suggestion="Check device connection and try again"
                )
            )
        except DeviceConnectionError as e:
            logger.error(f"Device connection error: {str(e)}")
            return JSONResponse(
                status_code=503,
                content=error_response(
                    message=str(e),
                    error_type="DeviceConnectionError",
                    suggestion="Ensure device is properly connected"
                )
            )
        except Exception as e:
            logger.error(f"Unhandled exception in device operation: {str(e)}")
            return JSONResponse(
                status_code=500,
                content=error_response(
                    message="An unexpected error occurred",
                    error_type="InternalServerError"
                )
            )
    return wrapper

## 9. Component Standardization

### 9.1 Service Components

In [ ]:
class YourFeatureService:
    """Service for managing your feature operations"""
    
    def __init__(self, config=None):
        self.config = config or {}
        self.logger = logging.getLogger(__name__)
        # Initialize dependencies
        
    async def operation_one(self, param1, param2):
        """Perform operation one"""
        # Implementation...
        
    async def operation_two(self, param1, param2):
        """Perform operation two"""
        # Implementation...

### 9.2 Logger Standardization

In [ ]:
def setup_logger(name, log_level=logging.INFO):
    """Setup standardized logger"""
    logger = logging.getLogger(name)
    logger.setLevel(log_level)
    
    # Create handler with formatting
    handler = logging.StreamHandler()
    formatter = logging.Formatter(
        '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
    )
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    
    return logger

## 10. Frontend Integration

### 10.1 Backend Requirements for Frontend

When creating backend functionality, consider frontend requirements:

1. Clear API documentation for frontend developers
2. Consistent response structure
3. Appropriate HTTP status codes
4. CORS configuration for development
5. WebSocket event standards for real-time updates

### 10.2 API Documentation

Generate comprehensive API documentation:

In [ ]:
@app.get("/api/docs.json")
async def get_api_docs():
    """Generate API documentation for frontend"""
    routes = []
    categorized_routes = {}
    
    for route in app.routes:
        if hasattr(route, "methods") and getattr(route, "path", None):
            route_info = {
                "path": route.path,
                "method": list(route.methods)[0].lower(),
                "description": route.description or "",
                "tags": getattr(route, "tags", []),
                "parameters": []
            }
            
            # Add to main list
            routes.append(route_info)
            
            # Categorize by tags
            for tag in route_info.get("tags", ["other"]):
                if tag not in categorized_routes:
                    categorized_routes[tag] = []
                categorized_routes[tag].append(route_info)
    
    return {
        "routes": routes,
        "categorized_routes": categorized_routes,
        "routes_count": len(routes)
    }

## 11. Testing & Debugging

### 11.1 Unit Testing

In [ ]:
# tests/test_your_feature.py
import pytest
from fastapi.testclient import TestClient
from app import app

client = TestClient(app)

def test_get_your_feature():
    """Test GET /api/your-feature endpoint"""
    response = client.get("/api/your-feature")
    assert response.status_code == 200
    data = response.json()
    assert "status" in data
    assert data["status"] == "success"

def test_create_your_feature():
    """Test POST /api/your-feature endpoint"""
    response = client.post(
        "/api/your-feature",
        json={"parameter1": "test", "parameter2": 123}
    )
    assert response.status_code == 200
    data = response.json()
    assert "status" in data
    assert data["status"] == "success"

### 11.2 Debugging Helpers

In [ ]:
@app.get("/debug/status")
async def debug_status():
    """Debugging endpoint to check system status"""
    # Collect system information
    status_info = {
        "readers": await get_reader_status(),
        "websockets": get_websocket_connections(),
        "system": {
            "memory_usage": get_memory_usage(),
            "cpu_usage": get_cpu_usage(),
            "uptime": get_uptime()
        }
    }
    return status_info

## 12. Feature Implementation Checklist

### 12.1 Initial Planning

- [ ] Define feature requirements and specifications
- [ ] Identify hardware dependencies
- [ ] Plan API endpoints and WebSocket events
- [ ] Design data models and database schema (if needed)
- [ ] Identify frontend integration requirements

### 12.2 Backend Implementation

- [ ] Create necessary Python modules and classes
- [ ] Implement API endpoints with proper validation
- [ ] Develop WebSocket functionality (if needed)
- [ ] Implement hardware integration
- [ ] Add error handling and logging
- [ ] Document API endpoints and WebSocket events

### 12.3 Testing & Validation

- [ ] Write unit tests for core functionality
- [ ] Test API endpoints with real and simulated data
- [ ] Verify WebSocket communication
- [ ] Test hardware integration
- [ ] Validate error handling scenarios

## 13. Current Status

### 13.1 Completed Features

- ✅ Core FastAPI application structure
- ✅ RESTful API endpoint framework
- ✅ WebSocket implementation for real-time updates
- ✅ Smart card reader integration
- ✅ NFC operations support
- ✅ UWB positioning framework
- ✅ Hardware simulation mode
- ✅ Error handling middleware
- ✅ Logging system
- ✅ Configuration management
- ✅ API documentation endpoints

### 13.2 Pending Developments

- ⬜ Authentication and authorization system
- ⬜ Comprehensive database integration
- ⬜ Full unit test coverage
- ⬜ Advanced websocket message queuing
- ⬜ Complete hardware error recovery system
- ⬜ Plugin system for extensibility
- ⬜ Advanced monitoring dashboard
- ⬜ Rate limiting and security middleware
- ⬜ Complete biometric integration
- ⬜ Production deployment configuration

## 14. Best Practices

1. Follow established patterns in existing code
2. Use async/await consistently for I/O operations
3. Document all functions, classes, and API endpoints
4. Implement proper error handling with specific exceptions
5. Use logging instead of print statements
6. Keep code modular and follow single responsibility principle
7. Test with both real and simulated hardware
8. Validate all input data using Pydantic models
9. Use type hints consistently
10. Use dependency injection where appropriate

## 15. Additional Resources

- [FastAPI Documentation](https://fastapi.tiangolo.com/)
- [Async/Await in Python](https://docs.python.org/3/library/asyncio.html)
- [WebSockets with FastAPI](https://fastapi.tiangolo.com/advanced/websockets/)
- [Pydantic Documentation](https://pydantic-docs.helpmanual.io/)
- Internal Documentation:
  - [Frontend Development Guide](/documentation/frontenddevelopment.md)
  - [WebSocket Guide](/documentation/websocketguide.md)
  - [Full Stack Overview](/documentation/fullstackoverview.md)